In [ ]:
# only need to run once per session
!pip install -q -U keras-tuner


In [ ]:
!pip install -q gdown

#Use the h5 file from my drive
file_id = "1-1lCaTKdlHjUBVvdHuUJTaRimrINcdXS"
!gdown --id $file_id -O best_model.h5

# confirm
!ls -lh best_model.h5


In [ ]:
# Fashion-MNIST Extra Credit Notebook
# Requirements:
# - >98% validation accuracy on Fashion-MNIST
# - Checkpoints, EarlyStopping, Dropout, Hyperparameter tuning, TensorBoard
# - Save final model as .h5 and load it

# 1. Install dependencies
!pip install -q -U keras-tuner visualkeras

# 2. Enable TensorBoard
%load_ext tensorboard

# 3. Imports
import os, datetime
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks
import keras_tuner as kt

model = keras.models.load_model("best_model.h5")

# 4. Load & preprocess data
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
# split off 5k for validation
x_train, x_val = x_train[:-5000], x_train[-5000:]
y_train, y_val = y_train[:-5000], y_train[-5000:]
# normalize & reshape
x_train = x_train.astype("float32")/255.0
x_val   = x_val.astype("float32")/255.0
x_test  = x_test.astype("float32")/255.0
x_train = x_train[..., None]
x_val   = x_val[..., None]
x_test  = x_test[..., None]

# 5. Define hypermodel builder
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Input((28,28,1)))
    # Conv block 1
    model.add(layers.Conv2D(
        hp.Choice('filters1',[32,64,96]), 3, padding='same', activation='relu'
    ))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    # Conv block 2
    model.add(layers.Conv2D(
        hp.Choice('filters2',[64,128,192]), 3, padding='same', activation='relu'
    ))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D())
    # Dense head
    model.add(layers.Flatten())
    model.add(layers.Dropout(hp.Float('dropout1',0.2,0.5,step=0.1)))
    model.add(layers.Dense(
        hp.Int('dense_units', 128, 512, step=128), activation='relu'
    ))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(hp.Float('dropout2',0.2,0.5,step=0.1)))
    model.add(layers.Dense(10, activation='softmax'))
    # Compile
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',[1e-2,1e-3,1e-4])
        ),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# 6. Set up KerasTuner (Hyperband)
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='kt_dir',
    project_name='fashion_extra'
)
# callbacks for tuning
stop_early = callbacks.EarlyStopping(monitor='val_loss', patience=3)
log_dir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
tb_tuner = callbacks.TensorBoard(log_dir=log_dir)

# 7. Run hyperparameter search
tuner.search(
    x_train, y_train,
    epochs=10,
    validation_data=(x_val, y_val),
    callbacks=[stop_early, tb_tuner]
)

# 8. Build the best model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)

# 9. Final training with checkpoints & early stopping
checkpoint_cb = callbacks.ModelCheckpoint(
    'best_model.h5', save_best_only=True, monitor='val_accuracy'
)
early_cb = callbacks.EarlyStopping(
    monitor='val_accuracy', patience=5, restore_best_weights=True
)
history = model.fit(
    x_train, y_train,
    epochs=30,
    validation_data=(x_val, y_val),
    callbacks=[checkpoint_cb, early_cb, tb_tuner]
)

# 10. Load and evaluate the saved model
loaded = keras.models.load_model('best_model.h5')
loss, acc = loaded.evaluate(x_test, y_test, verbose=0)
print(f'Test accuracy: {acc*100:.2f}%')

# 11. Launch TensorBoard
%tensorboard --logdir logs
